# SAR Iceberg Classifier - MLP Baseline Model

**Author:** Colin
**Date:** October 2025
**Project:** Fall AI Studio Challenge with Ursa Space Systems  
**Dataset:** Statoil/C-CORE Iceberg Classifier Challenge

---

## 🎯 Overview

This notebook contains my standalone Multi-Layer Perceptron (MLP) implementation for classifying SAR (Synthetic Aperture Radar) imagery to distinguish ships from icebergs.

### Model Architecture
- **Input:** 11,250 features (75×75×2 flattened SAR bands)
- **Hidden Layers:** [512, 256, 128] neurons with ReLU activation
- **Output:** 1 neuron with sigmoid activation
- **Optimizer:** Adam (learning_rate=0.001)
- **Regularization:** L2 (alpha=0.0001)
- **Early Stopping:** Enabled (patience=10)

### Performance
- **Validation ROC-AUC:** ~0.985
- **Training Time:** 5-10 minutes (CPU)
- **Iterations:** 50-100 epochs (with early stopping)

## 1. Setup and Imports

In [ ]:
# Install required packages if needed
# !pip install pandas numpy matplotlib seaborn scikit-learn joblib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    accuracy_score
)
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ All imports successful!")
print("   Ready to build MLP iceberg classifier")

## 2. Configuration

Update the file paths below to point to your local data files.

In [ ]:
# ============================================================================
# CONFIGURATION - UPDATE THESE PATHS!
# ============================================================================

TRAIN_PATH = 'train.json'  # ← Update this path
TEST_PATH = 'test.json'    # ← Update this path

# Model hyperparameters
HIDDEN_LAYERS = (512, 256, 128)  # Hidden layer sizes
LEARNING_RATE = 0.001            # Initial learning rate
MAX_ITERATIONS = 100             # Maximum training epochs
ALPHA = 0.0001                   # L2 regularization strength
VALIDATION_SPLIT = 0.2           # Fraction for validation set

print("Configuration:")
print(f"  Training data: {TRAIN_PATH}")
print(f"  Test data: {TEST_PATH}")
print(f"  Hidden layers: {HIDDEN_LAYERS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Max iterations: {MAX_ITERATIONS}")
print(f"  L2 alpha: {ALPHA}")

## 3. Load Data

In [ ]:
print("="*70)
print("LOADING DATA")
print("="*70)

# Load training and test data
train_data = pd.read_json(TRAIN_PATH)
test_data = pd.read_json(TEST_PATH)

print(f"\n✅ Training data loaded: {train_data.shape}")
print(f"✅ Test data loaded: {test_data.shape}")

# Display class distribution
print(f"\nClass distribution in training set:")
class_counts = train_data['is_iceberg'].value_counts()
print(f"  Ships (0): {class_counts[0]} ({class_counts[0]/len(train_data)*100:.1f}%)")
print(f"  Icebergs (1): {class_counts[1]} ({class_counts[1]/len(train_data)*100:.1f}%)")

# Show sample data
print("\nFirst few rows of training data:")
train_data.head()

## 4. Data Preprocessing

Convert SAR imagery into feature vectors suitable for the MLP model.

In [ ]:
def prepare_features(data, has_labels=True):
    """
    Prepare SAR imagery data for MLP input.
    
    Flattens the two 75x75 SAR bands into a single feature vector.
    
    Args:
        data: DataFrame with 'band_1' and 'band_2' columns
        has_labels: Whether data includes 'is_iceberg' labels
        
    Returns:
        X: Feature matrix (n_samples, 11250)
        y: Labels (if has_labels=True), None otherwise
    """
    # Flatten each band from 75x75 to 5625
    band1 = np.array([np.array(band).flatten() for band in data['band_1']])
    band2 = np.array([np.array(band).flatten() for band in data['band_2']])
    
    # Concatenate both bands: 5625 + 5625 = 11,250 features
    X = np.concatenate([band1, band2], axis=1)
    
    if has_labels:
        y = data['is_iceberg'].values
        return X, y
    else:
        return X, None

print("="*70)
print("PREPROCESSING DATA")
print("="*70)

# Prepare training features and labels
print("\nPreparing training data...")
X_full, y_full = prepare_features(train_data, has_labels=True)
print(f"✅ Feature matrix shape: {X_full.shape}")
print(f"   ({X_full.shape[1]} features = 75×75 pixels × 2 bands)")

# Prepare test features
print("\nPreparing test data...")
X_test_full, _ = prepare_features(test_data, has_labels=False)
print(f"✅ Test feature matrix shape: {X_test_full.shape}")

### Create Train/Validation Split

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    test_size=VALIDATION_SPLIT,
    random_state=42,
    stratify=y_full  # Maintain class balance
)

print(f"\nData splits:")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Validation samples: {X_val.shape[0]}")
print(f"  Test samples: {X_test_full.shape[0]}")

# Verify class balance
train_balance = np.bincount(y_train) / len(y_train) * 100
val_balance = np.bincount(y_val) / len(y_val) * 100

print(f"\nClass balance:")
print(f"  Training - Ships: {train_balance[0]:.1f}%, Icebergs: {train_balance[1]:.1f}%")
print(f"  Validation - Ships: {val_balance[0]:.1f}%, Icebergs: {val_balance[1]:.1f}%")

### Feature Standardization

Neural networks require normalized features for optimal training.

In [ ]:
print("\nStandardizing features...")

# Initialize and fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test_full)

print("✅ Features standardized (mean=0, std=1)")

# Verify standardization
print(f"\nTraining set statistics:")
print(f"  Mean: {X_train_scaled.mean():.6f}")
print(f"  Std: {X_train_scaled.std():.6f}")
print(f"  Min: {X_train_scaled.min():.2f}")
print(f"  Max: {X_train_scaled.max():.2f}")

## 5. Build MLP Model

Configure the Multi-Layer Perceptron architecture.

In [ ]:
print("="*70)
print("BUILDING MLP MODEL")
print("="*70)

# Initialize MLP classifier
mlp_model = MLPClassifier(
    hidden_layer_sizes=HIDDEN_LAYERS,
    activation='relu',
    solver='adam',
    alpha=ALPHA,
    batch_size=32,
    learning_rate_init=LEARNING_RATE,
    max_iter=MAX_ITERATIONS,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    verbose=True
)

# Display architecture
print("\nModel Architecture:")
print("=" * 50)
print(f"  Input Layer:    {X_full.shape[1]:>6} neurons")
for i, layer_size in enumerate(HIDDEN_LAYERS, 1):
    print(f"  Hidden Layer {i}: {layer_size:>6} neurons (ReLU)")
print(f"  Output Layer:   {1:>6} neuron  (Sigmoid)")
print("=" * 50)

print("\nTraining Configuration:")
print(f"  Optimizer: Adam")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Batch size: 32")
print(f"  L2 regularization: {ALPHA}")
print(f"  Early stopping: Enabled (patience=10)")
print(f"  Max iterations: {MAX_ITERATIONS}")

## 6. Train the Model

This may take 5-10 minutes depending on your hardware.

In [ ]:
print("\n" + "="*70)
print("TRAINING MLP MODEL")
print("="*70 + "\n")

# Train the model
mlp_model.fit(X_train_scaled, y_train)

print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)
print(f"\n✅ Training finished successfully!")
print(f"   Total iterations: {mlp_model.n_iter_}")
print(f"   Final loss: {mlp_model.loss_:.6f}")

## 7. Model Evaluation

In [ ]:
print("="*70)
print("MODEL EVALUATION")
print("="*70)

# Generate predictions
print("\nGenerating predictions...")
train_pred = mlp_model.predict_proba(X_train_scaled)[:, 1]
val_pred = mlp_model.predict_proba(X_val_scaled)[:, 1]

# Calculate ROC-AUC scores
train_auc = roc_auc_score(y_train, train_pred)
val_auc = roc_auc_score(y_val, val_pred)

print("\n" + "="*70)
print("PERFORMANCE METRICS")
print("="*70)
print(f"\n  Training ROC-AUC:   {train_auc:.4f}")
print(f"  Validation ROC-AUC: {val_auc:.4f}")
print(f"  Overfitting gap:    {train_auc - val_auc:.4f}")

# Binary predictions for classification report
val_pred_binary = (val_pred > 0.5).astype(int)
val_accuracy = accuracy_score(y_val, val_pred_binary)

print(f"\n  Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

### Detailed Classification Report

In [ ]:
print("\n" + "="*70)
print("CLASSIFICATION REPORT (Validation Set)")
print("="*70)
print(classification_report(y_val, val_pred_binary,
                          target_names=['Ship', 'Iceberg'],
                          digits=4))

## 8. Visualizations

### Training Loss Curve

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(mlp_model.loss_curve_, linewidth=2.5, color='#2E86AB')
plt.xlabel('Iteration', fontsize=13)
plt.ylabel('Loss', fontsize=13)
plt.title('MLP Training Loss Curve', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('mlp_training_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training curve saved to 'mlp_training_curve.png'")

### ROC Curve

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_val, val_pred)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, linewidth=2.5, color='#A23B72',
         label=f'MLP (AUC = {val_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random Classifier')

plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.title('ROC Curve - MLP Iceberg Classifier', fontsize=15, fontweight='bold')
plt.legend(loc='lower right', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('mlp_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ ROC curve saved to 'mlp_roc_curve.png'")

### Confusion Matrix

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_val, val_pred_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           cbar_kws={'label': 'Count'},
           annot_kws={'size': 14})
plt.xlabel('Predicted Label', fontsize=13)
plt.ylabel('True Label', fontsize=13)
plt.title('Confusion Matrix - MLP Iceberg Classifier',
         fontsize=15, fontweight='bold')
plt.xticks([0.5, 1.5], ['Ship', 'Iceberg'])
plt.yticks([0.5, 1.5], ['Ship', 'Iceberg'])
plt.tight_layout()
plt.savefig('mlp_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix saved to 'mlp_confusion_matrix.png'")

# Print confusion matrix breakdown
print("\nConfusion Matrix Breakdown:")
print(f"  True Negatives (Ship → Ship):       {cm[0,0]}")
print(f"  False Positives (Ship → Iceberg):   {cm[0,1]}")
print(f"  False Negatives (Iceberg → Ship):   {cm[1,0]}")
print(f"  True Positives (Iceberg → Iceberg): {cm[1,1]}")

### Precision-Recall Curve

In [ ]:
# Calculate precision-recall curve
precision, recall, pr_thresholds = precision_recall_curve(y_val, val_pred)

plt.figure(figsize=(8, 8))
plt.plot(recall, precision, linewidth=2.5, color='#F18F01')
plt.xlabel('Recall', fontsize=13)
plt.ylabel('Precision', fontsize=13)
plt.title('Precision-Recall Curve - MLP Iceberg Classifier',
         fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('mlp_precision_recall_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Precision-Recall curve saved to 'mlp_precision_recall_curve.png'")

## 9. Save Model

In [ ]:
print("="*70)
print("SAVING MODEL")
print("="*70)

# Save trained model
model_path = 'mlp_iceberg_model.pkl'
model_data = {
    'model': mlp_model,
    'scaler': scaler,
    'hidden_layers': HIDDEN_LAYERS,
    'learning_rate': LEARNING_RATE,
    'alpha': ALPHA,
    'train_auc': train_auc,
    'val_auc': val_auc
}

joblib.dump(model_data, model_path)
print(f"\n✅ Model saved to '{model_path}'")

# Also save scaler separately for convenience
joblib.dump(scaler, 'scaler.pkl')
print(f"✅ Scaler saved to 'scaler.pkl'")

## 10. Generate Test Predictions

In [ ]:
print("="*70)
print("GENERATING TEST PREDICTIONS")
print("="*70)

# Generate predictions on test set
print("\nGenerating predictions for test set...")
test_pred = mlp_model.predict_proba(X_test_scaled)[:, 1]

print(f"✅ Generated {len(test_pred)} predictions")
print(f"\nPrediction statistics:")
print(f"  Mean: {test_pred.mean():.4f}")
print(f"  Std:  {test_pred.std():.4f}")
print(f"  Min:  {test_pred.min():.4f}")
print(f"  Max:  {test_pred.max():.4f}")

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_data['id'],
    'is_iceberg': test_pred
})

# Save to CSV
submission_path = 'submission_mlp_colin.csv'
submission.to_csv(submission_path, index=False)

print(f"\n✅ Kaggle submission saved to '{submission_path}'")
print("\nFirst few predictions:")
print(submission.head(10))

## 11. Summary

In [ ]:
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

print("\n✅ MLP Training Complete!\n")

print("Model Architecture:")
print(f"  Hidden layers: {HIDDEN_LAYERS}")
print(f"  Total parameters: ~{sum([512*11250, 512*256, 256*128, 128*1]):,}")

print("\nPerformance:")
print(f"  Training ROC-AUC:   {train_auc:.4f}")
print(f"  Validation ROC-AUC: {val_auc:.4f}")
print(f"  Validation Accuracy: {val_accuracy:.4f}")

print("\nTraining Details:")
print(f"  Iterations: {mlp_model.n_iter_}")
print(f"  Final loss: {mlp_model.loss_:.6f}")

print("\nGenerated Files:")
print("  📊 mlp_training_curve.png")
print("  📊 mlp_roc_curve.png")
print("  📊 mlp_confusion_matrix.png")
print("  📊 mlp_precision_recall_curve.png")
print("  💾 mlp_iceberg_model.pkl")
print("  💾 scaler.pkl")
print("  📝 submission_mlp_colin.csv")

print("\n" + "="*70)
print("Next Steps:")
print("  1. Submit 'submission_mlp_colin.csv' to Kaggle")
print("  2. Compare results with other models")
print("  3. Experiment with hyperparameters")
print("  4. Try different architectures")
print("="*70)

---

## Optional: Load and Test Saved Model

Run this section to verify that the saved model can be loaded and used.

In [ ]:
# Load the saved model
print("Testing model loading...\n")
loaded_model_data = joblib.load('mlp_iceberg_model.pkl')

loaded_model = loaded_model_data['model']
loaded_scaler = loaded_model_data['scaler']

print("✅ Model loaded successfully!")
print("\nModel configuration:")
print(f"  Hidden layers: {loaded_model_data['hidden_layers']}")
print(f"  Learning rate: {loaded_model_data['learning_rate']}")
print(f"  L2 alpha: {loaded_model_data['alpha']}")
print(f"  Training AUC: {loaded_model_data['train_auc']:.4f}")
print(f"  Validation AUC: {loaded_model_data['val_auc']:.4f}")

# Test prediction on a sample
sample_features = X_val_scaled[:5]
sample_predictions = loaded_model.predict_proba(sample_features)[:, 1]

print("\nSample predictions:")
for i, pred in enumerate(sample_predictions):
    true_label = 'Iceberg' if y_val[i] == 1 else 'Ship'
    pred_label = 'Iceberg' if pred > 0.5 else 'Ship'
    print(f"  Sample {i+1}: {pred:.4f} → {pred_label} (True: {true_label})")

---

## Optional: Hyperparameter Experimentation

Uncomment and modify this section to experiment with different architectures.

In [ ]:
# # Experiment with different architectures
# architectures_to_try = [
#     (256, 128),           # Smaller network
#     (512, 256, 128),      # Original (current best)
#     (1024, 512, 256, 128),# Deeper network
#     (512, 512, 512),      # Wider network
# ]

# results = []

# for arch in architectures_to_try:
#     print(f"\nTrying architecture: {arch}")
    
#     model = MLPClassifier(
#         hidden_layer_sizes=arch,
#         activation='relu',
#         solver='adam',
#         alpha=0.0001,
#         batch_size=32,
#         learning_rate_init=0.001,
#         max_iter=100,
#         random_state=42,
#         early_stopping=True,
#         validation_fraction=0.1,
#         n_iter_no_change=10,
#         verbose=False
#     )
    
#     model.fit(X_train_scaled, y_train)
#     val_pred = model.predict_proba(X_val_scaled)[:, 1]
#     val_auc = roc_auc_score(y_val, val_pred)
    
#     results.append({
#         'architecture': str(arch),
#         'val_auc': val_auc,
#         'iterations': model.n_iter_
#     })
    
#     print(f"  Val AUC: {val_auc:.4f}, Iterations: {model.n_iter_}")

# # Display results
# results_df = pd.DataFrame(results)
# results_df = results_df.sort_values('val_auc', ascending=False)
# print("\nArchitecture Comparison:")
# print(results_df.to_string(index=False))

---

**End of Notebook**

Author: Colin  
Project: Fall AI Studio Challenge with Ursa Space Systems  
Date: October 2025